In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.constraints import min_max_norm
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.initializers import GlorotNormal

from envModel import EnvDense, EnvConv2D
import sys
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

In [ ]:
# nn
batch_size = 128
epochs = 10
use_bias = True
kernel_init = GlorotNormal()
loss_function = 'categorical_crossentropy' #'squared_hinge'

# learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

In [ ]:
#Load the data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print(X_train.shape, 'train samples')
print(X_test.shape, 'test samples')

In [ ]:
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

#one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10) # * 2 - 1
y_test = tf.keras.utils.to_categorical(y_test, 10) # * 2 - 1

In [ ]:
model = Sequential()

model.add(EnvConv2D(32,8, kernel_size=(5,5), kernel_initializer=kernel_init,
                    input_shape=X_train.shape[1:]))
model.add(BatchNormalization(epsilon=1e-5, momentum=0.99))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(EnvConv2D(64,8, kernel_size=(5,5), kernel_initializer=kernel_init))
model.add(BatchNormalization(epsilon=1e-5, momentum=0.99))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(EnvDense(512,8, kernel_initializer=kernel_init))
model.add(BatchNormalization(epsilon=1e-5, momentum=0.99))
model.add(Activation('relu'))

model.add(EnvDense(10,8, kernel_initializer=kernel_init))
model.add(BatchNormalization(epsilon=1e-5, momentum=0.99))
model.add(Activation('softmax'))

# Printing the model
print(model.summary())

opt = Adam(lr=lr_start)

In [ ]:
# Step 3: Compile the Model
model.compile(loss=loss_function,optimizer=opt,metrics=['accuracy'])

lr_scheduler = LearningRateScheduler(lambda e: lr_start * lr_decay ** e)

In [ ]:
# Step 4: Train the Model

history = model.fit(X_train,y_train,epochs=epochs,batch_size=128,
                    validation_data=(X_test,y_test),verbose=1,callbacks=[lr_scheduler])

In [ ]:
import sys
sys.path.insert(1, r"C:\Users\basar\Desktop\MSDegree\Thesis\Experiments\QuantCode")
import Utils as uts
import matplotlib.pyplot as plt
uts.plotHistory(history)

In [ ]:
# Step 5: Evaluate the Model
loss,accuracy = model.evaluate(X_test,y_test)
print("Loss = ",loss)
print("Accuracy = ",accuracy)

In [ ]:
model.save_weights('mnistBase.h5')

In [ ]:
def getStates(model, curInd, inds):
    states = []
    states.append(curInd)
    for i in inds:
        l = model.get_layer(index = i)
        states.append(l.getK())
    return states

In [ ]:
inds = [0,4,9,12]
x = getStates(model,9,inds)

In [ ]:
x

In [ ]:
#construct state space
states = []
for currLayer in range(4):
    for l0 in range(2,8):
        for l1 in range(2,8):
            for l2 in range(2,8):
                for l3 in range(2,8):
                    states.append((currLayer,l0,l1,l2,l3))

Q = {}
for s in states:
    for a in range(2,8):
        Q[s, a] = 0

In [ ]:
l = model.get_layer(index=9)

In [ ]:
l.get_weights()[0].shape

In [ ]:
x = l.get_weights()[0].shape

In [ ]:
x[1]

In [ ]:
import numpy as np
a = np.squeeze(np.shape(x))
print(a)

In [ ]:
if a==4:
    print('asd')

In [ ]:
l1 = model.get_layer(name='env_conv2d')
l1.setK(4)
l1 = model.get_layer(name='env_conv2d_1')
l1.setK(4)
l1 = model.get_layer(name='env_dense')
l1.setK(5)
l1 = model.get_layer(name='env_dense_1')
l1.setK(7)

In [ ]:
model.compile(loss=loss_function,optimizer=opt,metrics=['accuracy'])

In [ ]:
def clip(x):
    return tf.math.maximum(-1.,tf.math.minimum(1.,x))

def quantize(ri, k):
    level = tf.math.pow(2., k) - 1.
    ro = (1. / level) * tf.math.round(level * ri)
    return ri + tf.stop_gradient(ro - ri)
def quantOpt(x):
    xc = clip(x)
    xq = quantize(xc, k=4)
    return xq

In [ ]:
rn = -1
uts.plotLayer(model,'env_conv2d',rng=rn)
uts.plotLayer(model,'env_conv2d_1',rng=rn)
uts.plotLayer(model,'env_dense',rng=rn)
uts.plotLayer(model,'env_dense_1',rng=rn)

In [ ]:
import QuantCodes as qc
def setwgt(name):
    c1 = model.get_layer(name=name).get_weights()    
    wb = quantOpt(c1[0])    
    c1[0] = wb
    model.get_layer(name=name).set_weights(c1)

In [ ]:
setwgt('env_conv2d')
setwgt('env_conv2d_1')
setwgt('env_dense')
setwgt('env_dense_1')

In [ ]:
rn = -1
uts.plotLayer(model,'env_conv2d',rng=rn)
uts.plotLayer(model,'env_conv2d_1',rng=rn)
uts.plotLayer(model,'env_dense',rng=rn)
uts.plotLayer(model,'env_dense_1',rng=rn)

In [ ]:
# Step 5: Evaluate the Model
loss,accuracy = model.evaluate(X_test,y_test)
print("Loss = ",loss)
print("Accuracy = ",accuracy)

In [ ]:
from tensorflow.keras.models import Model
layer_name = 'activation'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(X_test)

In [ ]:
intermediate_output